In [1]:
import pathlib
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date
from sqlalchemy.orm import Session
from sqlalchemy import inspect, func, text
import altair as alt

# Create path object using pathlib
This will ensure this file will run for everyone regardless of os
This group contains folks running on mac, windows, and linux.    

In [2]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Project_3_Group_7':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [3]:
resources_path = pathlib.PurePath(cwd, 'Resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/Resources')

# Create the db session
Can not use automap_base because primary key is a composite key 

https://python-code.dev/articles/132521261

In [4]:
engine = sqlalchemy.create_engine(f'sqlite:///{resources_path}/data.sqlite')
Base = declarative_base()

/tmp/ipykernel_51276/3378198577.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

['steam_twitch_agg', 'tags', 'twitch_monthly']

In [6]:
steam_twitch_tags_table = tables[0]
tags_table  = tables[1]
twitch_monthly_table = tables[2]

In [7]:
class steam_twitch_agg(Base):
    __tablename__ = steam_twitch_tags_table
    # https://stackoverflow.com/questions/19129289/how-to-define-composite-primary-key-in-sqlalchemy
    # indicates that this syntax should make a composite primary key
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)
    title = Column(String(128))
    # These two will need to be changed to Float 
    hours_watched = Column(Float)
    hours_streamed = Column(Float)
    average_streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)
    release_date = Column(Date)
    reviews_total = Column(Integer)
    review_avg_percent = Column(Integer)
    launch_price_cents = Column(Integer)
    dataset_est_rev_cents = Column(Integer)

class twitch_monthly(Base):
    __tablename__ = twitch_monthly_table
    app_id = Column(Integer, primary_key=True)
    title = Column(String(128))
    rank = Column(Integer)
    month = Column(Integer, primary_key=True)
    year = Column(Integer, primary_key=True)
    hours_watched = Column(Integer)
    hours_streamed = Column(Integer)
    peak_viewers = Column(Integer)
    peak_channels = Column(Integer)
    streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)

class steam_tags(Base):
    __tablename__ = tags_table
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)


In [8]:
Base.metadata.create_all(engine)

In [9]:
session = Session(bind= engine)

# Queries

In [10]:
inner_statement = f"""SELECT st.app_id
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 3 
ORDER BY st.avg_hours_streamed DESC"""
statement = text(f"SELECT t.* FROM twitch_monthly AS t WHERE app_id IN ({inner_statement});")

result = session.execute(statement).all()
twtich_monthly_gte4_tags = result
print(len(twtich_monthly_gte4_tags))
twtich_monthly_gte4_tags

997


[(226, 214490, 'Alien: Isolation', 135, 1, 2016, 264294, 11799, 9590, 42, 3968, 355, 15),
 (227, 214490, 'Alien: Isolation', 188, 2, 2016, 109754, 7771, 10235, 35, 2653, 157, 11),
 (228, 214490, 'Alien: Isolation', 178, 4, 2016, 122248, 7052, 4976, 35, 2420, 170, 9),
 (229, 214490, 'Alien: Isolation', 186, 8, 2016, 121317, 7207, 3204, 29, 2292, 163, 9),
 (230, 214490, 'Alien: Isolation', 109, 10, 2016, 263311, 9398, 9862, 74, 2554, 354, 12),
 (231, 214490, 'Alien: Isolation', 182, 11, 2016, 121289, 7483, 2731, 70, 2008, 168, 10),
 (232, 214490, 'Alien: Isolation', 182, 12, 2016, 150761, 5866, 6323, 24, 387, 202, 7),
 (233, 214490, 'Alien: Isolation', 174, 6, 2017, 162443, 7350, 3835, 32, 2140, 225, 10),
 (234, 214490, 'Alien: Isolation', 142, 8, 2017, 277275, 7629, 14317, 33, 2186, 373, 10),
 (235, 214490, 'Alien: Isolation', 112, 10, 2017, 374932, 14409, 14512, 104, 3230, 504, 19),
 (236, 214490, 'Alien: Isolation', 162, 10, 2018, 300834, 17144, 4410, 137, 3456, 404, 23),
 (237, 21449

In [11]:
data = pd.DataFrame(twtich_monthly_gte4_tags)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             997 non-null    int64 
 1   app_id            997 non-null    int64 
 2   title             997 non-null    object
 3   rank              997 non-null    int64 
 4   month             997 non-null    int64 
 5   year              997 non-null    int64 
 6   hours_watched     997 non-null    int64 
 7   hours_streamed    997 non-null    int64 
 8   peak_viewers      997 non-null    int64 
 9   peak_channels     997 non-null    int64 
 10  streamers         997 non-null    int64 
 11  average_viewers   997 non-null    int64 
 12  average_channels  997 non-null    int64 
dtypes: int64(12), object(1)
memory usage: 101.4+ KB


# Clean datatypes to date

In [12]:
data = data[data['year'].gt(2018)]

In [13]:
data['yyyymm'] = data.year.astype(str).map(str) + data.month.astype(str).str.zfill(2)
data.head(3)

,index,app_id,title,rank,month,year,hours_watched,hours_streamed,peak_viewers,peak_channels,streamers,average_viewers,average_channels,yyyymm
11,237,214490,Alien: Isolation,126,1,2019,592800,10348,29261,46,2332,797,13,201901
12,238,214490,Alien: Isolation,195,2,2020,347691,9347,10613,39,1886,500,13,202002
13,239,214490,Alien: Isolation,180,5,2020,650974,53337,5911,273,9376,876,71,202005


In [14]:
data['date'] = pd.to_datetime(data['yyyymm'], format= '%Y%m')
data.head(3)

,index,app_id,title,rank,month,year,hours_watched,hours_streamed,peak_viewers,peak_channels,streamers,average_viewers,average_channels,yyyymm,date
11,237,214490,Alien: Isolation,126,1,2019,592800,10348,29261,46,2332,797,13,201901,2019-01-01
12,238,214490,Alien: Isolation,195,2,2020,347691,9347,10613,39,1886,500,13,202002,2020-02-01
13,239,214490,Alien: Isolation,180,5,2020,650974,53337,5911,273,9376,876,71,202005,2020-05-01


In [15]:
timeseries = data[["app_id", "date", "title", "hours_streamed"]]
timeseries

,app_id,date,title,hours_streamed
11,214490,2019-01-01,Alien: Isolation,10348
12,214490,2020-02-01,Alien: Isolation,9347
13,214490,2020-05-01,Alien: Isolation,53337
14,214490,2020-06-01,Alien: Isolation,31177
15,214490,2020-08-01,Alien: Isolation,30768
...,...,...,...,...
992,699130,2019-04-01,World War Z,132697
993,699130,2019-05-01,World War Z,60933
994,699130,2020-03-01,World War Z,92461
995,699130,2020-04-01,World War Z,105727


In [21]:
statement = text(f"""SELECT st.app_id
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 3
ORDER BY st.avg_hours_streamed DESC
LIMIT 7;""")
result = session.execute(statement).all()
top10_h_streamed = [row[0] for row in result]
top10_h_streamed

[10, 730, 1085660, 578080, 1174180, 924970, 1286680]

# Make timeseries plot

In [22]:
top_10_df = timeseries[timeseries['app_id'].isin(top10_h_streamed)]
top_10_df

,app_id,date,title,hours_streamed
79,924970,2020-12-01,Back 4 Blood,23282
80,924970,2021-08-01,Back 4 Blood,390022
81,924970,2021-10-01,Back 4 Blood,622033
82,924970,2021-11-01,Back 4 Blood,200227
83,924970,2021-12-01,Back 4 Blood,96514
...,...,...,...,...
798,1174180,2023-07-01,Red Dead Redemption 2,283878
799,1174180,2023-08-01,Red Dead Redemption 2,300012
800,1174180,2023-09-01,Red Dead Redemption 2,257686
979,1286680,2022-03-01,Tiny Tina's Wonderlands,179454


In [23]:
lines = alt.Chart(top_10_df).mark_line().encode(x='date', y='hours_streamed', color='title')
lines

alt.Chart(...)

# Close session

In [19]:
session.close()